In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import tensorflow as tf
import tflearn

In [52]:
train = pd.read_csv("train.csv")

In [53]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [88]:
trainY = train['Survived']; trainY = tflearn.data_utils.to_categorical(trainY, 2)
trainX = train.drop('PassengerId', axis=1); trainX = trainX.drop('Survived', axis=1); trainX = trainX.drop('Name', axis=1)

In [89]:
trainX.head()


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,NaN,S


In [92]:
le = preprocessing.LabelEncoder()
# Fill N/a
trainX['Cabin'].fillna("None")

# Change Categorical value to numerical value
for column in ['Sex', 'Ticket', 'Embarked','Cabin']:
    trainX[column] = le.fit_transform(trainX[column])
    
print trainX.head()



   Pclass  Sex   Age  SibSp  Parch  Ticket     Fare  Cabin  Embarked
0       3    1  22.0      1      0     523   7.2500      0         3
1       1    0  38.0      1      0     596  71.2833     82         1
2       3    0  26.0      0      0     669   7.9250      0         3
3       1    0  35.0      1      0      49  53.1000     56         3
4       3    1  35.0      0      0     472   8.0500      0         3


In [114]:
"""
Logistic Regression
"""

# Define parameters for the model
learning_rate = 0.01
batch_size = len(trainX)
n_epochs = 25


trainX = trainX.as_matrix()

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [115]:
X = tf.placeholder(tf.float32, [batch_size, 9], name="profile")
Y = tf.placeholder(tf.float32, [batch_size, 2], name="label")

In [116]:
# Step 4: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.Variable(tf.random_normal(shape=[9, 1], stddev=0.01), name="weights")
b = tf.Variable(tf.zeros([1,2]), name="bias")

In [117]:
# Step 5: predict Y from X and w, b
# the model that returns probability distribution of possible label of the image
# through the softmax layer
# a batch_size x 2 tensor that represents the possibility of the digits
logits = tf.matmul(X, w) + b

In [118]:
# Step 6: define loss function
# use softmax cross entropy with logits as the loss function
# compute mean cross entropy, softmax is applied internally
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss = tf.reduce_mean(entropy) # computes the mean over examples in the batch

In [119]:
# Step 7: define training op
# using gradient descent with learning rate of 0.01 to minimize cost
optimizer = \
tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    n_batches = int(len(trainX)/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        X_batch, Y_batch = trainX, trainY
        sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch})